<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

注意：因為爬蟲本身有時候沒有正確抓到資料，所以要讀新資料的時候，請多執行這支程式數次，把球員資料補齊

In [1]:
import PitchType_InfoCollector
import pandas as pd
import numpy as np
import time
import os

In [2]:
csv_file_path = 'MLB_Player_Stats_CSV\\'
years = range(2017,2024)
player_type = 'batter'

read_mode = 'continue' #'new' or 'continue'

對於極少數的球員 (2024.03.18觀測到五個) ，baseball savant 網站欠缺他們的球種進階資料

In [3]:
#-------------------Load player data----------------------------
dfs = []
for year in years:
    csv_file_name = f'{csv_file_path}{year}_{player_type}_arranged.csv'
    df = pd.read_csv(csv_file_name)
    dfs.append(df[['player_id', 'player_name']])
big_df = pd.concat(dfs, ignore_index = True)
player_ids = list(set(big_df['player_id']))

if os.path.exists(f'{csv_file_path}pitch_type_{player_type}.csv'):
    old_df = pd.read_csv(f'{csv_file_path}pitch_type_{player_type}.csv')
    old_player_ids = set(old_df['player_id'])
    player_ids = list(set(player_ids).difference(old_player_ids))
    print(f'There are {len(old_df)} rows in the old DF.')
print(f'這次要抓{len(player_ids)}個球員')

t = len(player_ids)
if os.path.exists(f'{csv_file_path}pitch_type_pitcher.csv'):
    dfs = [old_df,]
else:
    dfs = []
t0 = time.time()
for i, player_id in enumerate(player_ids):
    player_name = big_df[big_df.player_id.eq(player_id)]['player_name'].tolist()[0].replace(' ','').split(',')
    last_name = player_name[0]; first_name = player_name[1]
    
    print(f'{i+1}/{t} player_id: {player_id}')
    if player_type == 'pitcher':
        X = PitchType_InfoCollector.pitcher(player_id, first_name, last_name)
    elif player_type == 'batter':
        X = PitchType_InfoCollector.batter(player_id, first_name, last_name)
        
    if len(X) > 0:
        df = pd.DataFrame(X)
        print(f'There are {len(df)} rows in this DF.')
        dfs.append(df)
    if i % 10 == 9:
        entire_PA_df = pd.concat(dfs, ignore_index = True)
        print(f'There are {len(entire_PA_df)} rows in the entire DF.')
        entire_PA_df.to_csv(f'{csv_file_path}pitch_type_{player_type}.csv', index=False)
        
    t_this = time.time()
    print(f'Process time: {(t_this-t0)/60:.2f} min')
 

There are 41997 rows in the old DF.
這次要抓5個球員
1/5 player_id: 683232
Mears, Nick
Process time: 0.33 min
2/5 player_id: 502085
Robertson, David
Process time: 0.52 min
3/5 player_id: 622250
Sborz, Josh
Process time: 0.80 min
4/5 player_id: 608597
Guzman, Ronald
Process time: 0.80 min
5/5 player_id: 624507
Jones, Ryder
Process time: 0.81 min


In [5]:
entire_PA_df = pd.concat(dfs, ignore_index = True)
entire_PA_df.to_csv(f'{csv_file_path}pitch_type_{player_type}.csv', index=False)
print(f'There are {len(entire_PA_df)} rows in the entire DF.')

There are 41997 rows in the entire DF.
